In [1]:
"""
Welcome to the S2AND tutorial!

We will cover a few aspects of the S2AND pipeline:

(1) Load a test dataset.
(2) Fit a pairwise model + bells & whistles.
(3) Fit a clusterer.
(4) Evaluate the pairwise model and the clusterer.
""";

In [ ]:
import os

os.environ["OMP_NUM_THREADS"] = "8"

import json
import copy
import argparse
import logging
import pickle
from typing import Dict, Any, Optional, List
from collections import defaultdict

import numpy as np
import pandas as pd

from s2and.data import PDData
from s2and.featurizer import featurize, FeaturizationInfo
from s2and.model import PairwiseModeler, Clusterer, FastCluster
from s2and.eval import pairwise_eval, cluster_eval, facet_eval
from s2and.consts import FEATURIZER_VERSION, DEFAULT_CHUNK_SIZE, PROJECT_ROOT_PATH
from s2and.file_cache import cached_path
from s2and.plotting_utils import plot_facets
from hyperopt import hp

In [2]:
# this is the random seed we used for the ablations table
random_seed = 42
# number of cpus to use
n_jobs = 4

In [3]:
# we're going to load the arnetminer dataset
# and assume that you have it already downloaded to the `S2AND/data/` directory

dataset_name = 'test'
DATA_DIR = os.path.join(PROJECT_ROOT_PATH, 'data')

pddata = PDData(
    papers=os.path.join(DATA_DIR, dataset_name + "_papers.json"),
    name=dataset_name,
    mode="train",  # can also be 'inference' if just predicting
    clusters=os.path.join(DATA_DIR, dataset_name + "_clusters.json"),
    train_pairs=None,  # in case you have predefined splits for the pairwise models
    val_pairs=None,
    test_pairs=None,
    train_pairs_size=100000,  # how many training pairs for the pairwise models?
    val_pairs_size=10000,
    test_pairs_size=10000,
    n_jobs=n_jobs,
    load_name_counts=False,
    random_seed=random_seed,
)

2022-10-10 19:15:38,114 - s2and - INFO - loading papers
2022-10-10 19:15:38,129 - s2and - INFO - loaded papers
2022-10-10 19:15:38,130 - s2and - INFO - loading clusters
2022-10-10 19:15:38,132 - s2and - INFO - loaded clusters, loading specter
2022-10-10 19:15:38,133 - s2and - INFO - loaded specter, loading cluster seeds
2022-10-10 19:15:38,134 - s2and - INFO - loaded cluster seeds
2022-10-10 19:15:38,135 - s2and - INFO - making paper to cluster id
2022-10-10 19:15:38,136 - s2and - INFO - made paper to cluster id
2022-10-10 19:15:38,137 - s2and - INFO - preprocessing papers
Preprocessing papers: 100%|██████████| 356/356 [00:00<00:00, 767.56it/s]
2022-10-10 19:15:38,730 - s2and - INFO - preprocessed papers


In [4]:
# to train the pairwise model, we define which feature categories to use
# here it is all of them
features_to_use = [
    "author_similarity",
    "venue_similarity",
    "year_diff",
    "title_similarity",
    "abstract_similarity",
]

# we store all the information about the features in this convenient wrapper
featurization_info = FeaturizationInfo(features_to_use=features_to_use, featurizer_version=FEATURIZER_VERSION)

# now we can actually go and get the pairwise training, val and test data
train, val, test = featurize(pddata, featurization_info, n_jobs=4, use_cache=False, chunk_size=DEFAULT_CHUNK_SIZE, nan_value=np.nan)  # type: ignore
X_train, y_train, nameless_X_train = train
X_val, y_val, nameless_X_val = val
X_test, y_test, nameless_X_test = test

2022-10-10 19:15:40,569 - s2and - INFO - featurizing train
2022-10-10 19:15:40,571 - s2and - INFO - Creating 214 pieces of work
2022-10-10 19:15:40,573 - s2and - INFO - Created pieces of work
2022-10-10 19:15:40,574 - s2and - INFO - Cached changed, doing 214 work in parallel
2022-10-10 19:15:40,695 - s2and - INFO - Work completed
2022-10-10 19:15:40,697 - s2and - INFO - Making numpy arrays for features and labels
2022-10-10 19:15:40,699 - s2and - INFO - Numpy arrays made
2022-10-10 19:15:40,701 - s2and - INFO - featurized train, featurizing val
2022-10-10 19:15:40,702 - s2and - INFO - Creating 28 pieces of work
2022-10-10 19:15:40,704 - s2and - INFO - Created pieces of work
2022-10-10 19:15:40,705 - s2and - INFO - Cached changed, doing 28 work in parallel
2022-10-10 19:15:40,823 - s2and - INFO - Work completed
2022-10-10 19:15:40,826 - s2and - INFO - Making numpy arrays for features and labels
2022-10-10 19:15:40,827 - s2and - INFO - Numpy arrays made
2022-10-10 19:15:40,829 - s2and - 

In [5]:
# now we define and fit the pairwise modelers
pairwise_modeler = PairwiseModeler(
    n_iter=25,  # number of hyperparameter search iterations
    estimator=None,  # this will use the default LightGBM classifier
    search_space=None,  # this will use the default LightGBM search space
    monotone_constraints=featurization_info.lightgbm_monotone_constraints,  # we use monotonicity constraints to make the model more sensible
    random_state=random_seed,
)
pairwise_modeler.fit(X_train, y_train, X_val, y_val)

100%|██████████| 25/25 [00:01<00:00, 12.97trial/s, best loss: -0.875] 


In [6]:
# now we can fit the clusterer itself
clusterer = Clusterer(
    featurization_info,
    pairwise_modeler.classifier,  # the actual pairwise classifier
    cluster_model=FastCluster(linkage='average'),  # average linkage agglomerative clustering
    search_space={"eps": hp.uniform("choice", 0, 1)},  # the hyperparemetrs for the clustering algorithm
    n_jobs=n_jobs,
    use_cache=False,
    random_state=random_seed,
)
clusterer.fit(pddata)

2022-10-10 19:15:46,048 - s2and - INFO - Fitting clusterer
2022-10-10 19:15:46,081 - s2and - INFO - Making 1 distance matrices
2022-10-10 19:15:46,082 - s2and - INFO - Initializing pairwise_probas
2022-10-10 19:15:46,083 - s2and - INFO - Pairwise probas initialized, starting making all pairs
2022-10-10 19:15:46,084 - s2and - INFO - Featurizing batch 0/1
2022-10-10 19:15:46,085 - s2and - INFO - Getting constraints
2022-10-10 19:15:46,086 - s2and - INFO - Creating 120 pieces of work
2022-10-10 19:15:46,089 - s2and - INFO - Created pieces of work
2022-10-10 19:15:46,089 - s2and - INFO - Cached changed, doing 120 work in parallel


Finished loading model, total used 1695 iterations


2022-10-10 19:15:46,209 - s2and - INFO - Work completed
2022-10-10 19:15:46,212 - s2and - INFO - Making numpy arrays for features and labels
2022-10-10 19:15:46,214 - s2and - INFO - Numpy arrays made
2022-10-10 19:15:46,215 - s2and - INFO - Making predict flags
2022-10-10 19:15:46,216 - s2and - INFO - Pairwise classification
2022-10-10 19:15:46,220 - s2and - INFO - Starting to make matrices
Writing matrices: 100%|██████████| 120/120 [00:00<00:00, 261735.04it/s]
2022-10-10 19:15:46,223 - s2and - INFO - 1 distance matrices made


100%|██████████| 25/25 [00:00<00:00, 406.12trial/s, best loss: -0.597]

2022-10-10 19:15:46,291 - s2and - INFO - Clusterer fit


In [7]:
# but how good are our models? 
# first, let's look at the quality of the pairwise evaluation
pairwise_metrics = pairwise_eval(
    X_test,
    y_test,
    pairwise_modeler,
    os.path.join(PROJECT_ROOT_PATH, "data", "tutorial_figures"),  # where to put the figures
    "tutorial_figures",  # what to call the figures
    featurization_info.get_feature_names(),
    skip_shap=False,  # if your model isn't a tree-based model, you should put True here and it will not make SHAP figures
)
print(pairwise_metrics)

C extension was not built during install!


LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
All-NaN slice encountered


{'AUROC': 1.0, 'Average Precision': 1.0, 'F1': 1.0, 'Precision': 1.0, 'Recall': 1.0}


In [8]:
# we can do the same thing for the clustering performance
cluster_metrics, b3_metrics_per_signature = cluster_eval(
    pddata,
    clusterer,
    split="test",  # which part of the data to evaluate on, can also be 'val'
    use_s2_clusters=False,  # set to true if you want to see how the old S2 system does
)
print(cluster_metrics)

2022-10-10 19:15:53,995 - s2and - INFO - Making 1 distance matrices
2022-10-10 19:15:53,996 - s2and - INFO - Initializing pairwise_probas
2022-10-10 19:15:53,997 - s2and - INFO - Pairwise probas initialized, starting making all pairs
2022-10-10 19:15:53,998 - s2and - INFO - Featurizing batch 0/1
2022-10-10 19:15:53,999 - s2and - INFO - Getting constraints
2022-10-10 19:15:54,000 - s2and - INFO - Creating 153 pieces of work
2022-10-10 19:15:54,002 - s2and - INFO - Created pieces of work
2022-10-10 19:15:54,005 - s2and - INFO - Cached changed, doing 153 work in parallel
2022-10-10 19:15:54,134 - s2and - INFO - Work completed
2022-10-10 19:15:54,136 - s2and - INFO - Making numpy arrays for features and labels
2022-10-10 19:15:54,138 - s2and - INFO - Numpy arrays made
2022-10-10 19:15:54,140 - s2and - INFO - Making predict flags
2022-10-10 19:15:54,141 - s2and - INFO - Pairwise classification
2022-10-10 19:15:54,146 - s2and - INFO - Starting to make matrices
Writing matrices: 100%|████████

{'B3 (P, R, F1)': (0.595, 1.0, 0.746), 'Pred bigger ratio (mean, count)': (1.71, 7), 'True bigger ratio (mean, count)': (nan, 0)}


Mean of empty slice.
invalid value encountered in double_scalars
